# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
import pickle
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer


In [2]:
'''Uncommenet below portion if the system config supports nltk.download
In my local machine this download was not supported and was taking forever. So, I downloaded the zipped file;
punkt, wordnet and average_perceptron_tagger, created a folder /User/anup/anaconda3/nltk_data/
and place 1) /Users/anup/anaconda3/nltk_data/corpora/wordnet/
           2) /Users/anup/anaconda3/nltk_data/tokenizers/punkt
           3) /Users/anup/anaconda3/nltk_data/taggers/averaged_perceptron_tagger/
'''
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')

'Uncommenet below portion if the system config supports nltk.download\nIn my local machine this download was not supported and was taking forever. So, I downloaded the zipped file;\npunkt, wordnet and average_perceptron_tagger, created a folder /User/anup/anaconda3/nltk_data/\nand place 1) /Users/anup/anaconda3/nltk_data/corpora/wordnet/\n           2) /Users/anup/anaconda3/nltk_data/tokenizers/punkt\n           3) /Users/anup/anaconda3/nltk_data/taggers/averaged_perceptron_tagger/\n'

In [3]:
# load data from database
database_file="../data/disaster_response.db"
table_name="disaster_response_table"
engine = create_engine('sqlite:///'+ database_file)
df = pd.read_sql_table(table_name,engine)
df.head()
# X,Y variable
X = df['message']
Y = df.iloc[:,4:]


In [8]:
# Checking values in the category columns
Y = df.iloc[:,4:]
for column in Y.columns:
    print(Y[column].unique())

[1 0 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


In [9]:
Y.groupby("related").count()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,188,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [4]:
#The related column has few values of 2. Our classifier has two Outputs: 0 and 1.
# So, replacing 2 values by the max value (which is 1)
Y['related']= Y['related'].map(lambda x: 1 if x == 2 else x)

In [5]:
Y.groupby("related").count()


,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,...,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Tokenize the text function
    
    Arguments:
        text: Text messages
    Output:
        clean_tokens: List of tokens for the text
    """
    
    '''Replace all urls with a urlplaceholder string'''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    '''Extract all the urls from the text '''
    urls = re.findall(url_regex, text)
    
    '''Replace url with a urlplaceholder '''
    for url in urls:
        text = text.replace(url, 'urlplaceholder')

    '''Extract the word tokens from the provided text'''
    tokens = nltk.word_tokenize(text)
    
    '''Lemmanitizer to remove inflectional and derivationally related forms of a word'''
    lemmatizer = nltk.WordNetLemmatizer()

    '''List of clean tokens'''
    clean_tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model= pipeline.fit(X_train, y_train)

### 5. Test your model

Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
#Prediciton for test sets
y_test_pred= model.predict(X_test)
#y_test.values.shape
# Print classification report on test data
print(classification_report(y_test.values, y_test_pred, target_names=Y.columns.values))
#X_test[4135]

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5067
               request       0.75      0.53      0.62      1086
                 offer       0.00      0.00      0.00        37
           aid_related       0.77      0.59      0.67      2792
          medical_help       0.59      0.24      0.34       548
      medical_products       0.66      0.32      0.43       346
     search_and_rescue       0.61      0.23      0.33       187
              security       0.33      0.02      0.04       141
              military       0.67      0.33      0.44       218
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.60      0.66       420
                  food       0.81      0.67      0.74       722
               shelter       0.74      0.52      0.61       574
              clothing       0.69      0.40      0.51       105
                 money       0.67      

/Users/anup/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anup/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anup/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anup/anaconda3/lib/python3.8/site-p

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Get model parameters
parameters = pipeline.get_params().keys()
print('The parameters in the pipeline are:', parameters)

param_grid = {'classifier__estimator__learning_rate': [0.1, 0.01, 0.005],
                  'classifier__estimator__n_estimators': [10, 40, 50],
               'features__text_pipeline__vect__max_features': (None, 5000, 10000),
            'features__text_pipeline__tfidf__use_idf': (True, False)}
cv = GridSearchCV(pipeline, param_grid=param_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

The parameters in the pipeline are: dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vec

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f93a0b4d5f0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.1, 0.01,
                                                                  0.005],
                         'classifier__estimator__n_estimators': [

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:
# Get the prediction values from the grid search cross validator
y_test_pred_ft = cv.predict(X_test)
# Print classification report on test data
print(classification_report(y_test.values, y_test_pred_ft, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5040
               request       0.89      0.31      0.46      1106
                 offer       0.00      0.00      0.00        34
           aid_related       0.77      0.38      0.51      2670
          medical_help       0.52      0.02      0.05       492
      medical_products       0.70      0.05      0.09       299
     search_and_rescue       1.00      0.03      0.07       172
              security       0.00      0.00      0.00       117
              military       1.00      0.02      0.04       211
           child_alone       0.00      0.00      0.00         0
                 water       0.78      0.48      0.59       421
                  food       0.78      0.71      0.74       709
               shelter       0.88      0.31      0.46       593
              clothing       0.83      0.14      0.24       105
                 money       0.88      

/Users/a343808/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/a343808/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/a343808/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/a343808/anaconda3/lib/pyth

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
'''Build a custom transformer which builds starting verb of the sentence'''

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
         # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [38]:
'''Build a new pipeline with custom transformer as a feature'''

pipeline_update = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])




In [39]:
'''Use updated pipeline to train the model'''
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model_update= pipeline_update.fit(X_train, y_train)

In [41]:
#Prediciton for test sets
y_test_pred= model_update.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_test_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4999
               request       0.77      0.55      0.64      1107
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.62      0.68      2737
          medical_help       0.65      0.30      0.41       538
      medical_products       0.61      0.31      0.41       335
     search_and_rescue       0.82      0.21      0.33       195
              security       0.24      0.04      0.08       112
              military       0.63      0.29      0.40       219
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.66      0.69       423
                  food       0.80      0.71      0.75       752
               shelter       0.78      0.52      0.63       599
              clothing       0.74      0.49      0.59       109
                 money       0.50      

### 9. Export your model as a pickle file

In [42]:
with open('classifier.pkl', 'wb') as file:
        pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.